# Out-of-State-Contributions: Data Importation and Preparation

In [1]:
from functools import reduce
import numpy as np
import pandas as pd

%load_ext jupyternotify

pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 500)
pd.options.display.float_format = "{:,.2f}".format # Format floats

<IPython.core.display.Javascript object>

Import [National Institute on Money in Politics](https://www.followthemoney.org/) API key.

In [2]:
nimp_key = open("nimp_api_key.txt", "r").readline()

## Import and format the data

Import and format contribution-level data from the [National Institute on Money in Politics](https://www.followthemoney.org/) for gubernatorial, state senate and state house candidates in 2018, 2014 and 2010.

Download and save each cycle's contributions data and concatenate the data into a single file.

In [3]:
!sh process_contribs.sh

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/joe/.wget-hsts'. HSTS will be disabled.
--2018-08-28 20:33:39--  https://www.followthemoney.org/aaengine/aafetch.php?dt=1&y=2018&c-exi=1&c-r-ot=G,S,H&gro=c-t-id,d-id&APIKey=7393ac8fa32733ae574c429362bce82a&mode=csv
Resolving www.followthemoney.org (www.followthemoney.org)... 69.144.32.182
Connecting to www.followthemoney.org (www.followthemoney.org)|69.144.32.182|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/csv]
Saving to: ‘data/raw/contributions_18.csv’

data/raw/contributi     [                 <=>]   1.32G   916KB/s    in 42m 23s 

2018-08-28 21:20:47 (544 KB/s) - ‘data/raw/contributions_18.csv’ saved [1417140174]

Will not apply HSTS. The HSTS database must be a regular and non-world-writable file.
ERROR: could not open HSTS store at '/home/joe/.wget-hsts'. HSTS will be disabled.
--2018-08-28 21:20:47--  http

Import the contributions data.

In [4]:
%%notify
contributions = pd.read_csv("data/raw/contributions.csv", usecols=["Candidate", "Election_Status", "Specific_Party", "Election_Jurisdiction", "Election_Year", "Office_Sought", "Contributor", "Amount", "Date", "In-State"], error_bad_lines=False)
contributions.columns = ["candidate", "election_status", "party", "state", "year", "office", "contributor", "amount", "date", "in_out_state"]
contributions.info()

<IPython.core.display.Javascript object>

/home/joe/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (54) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,candidate,election_status,party,state,year,office,contributor,amount,date,in_out_state
0,"RAUNER, BRUCE VINCENT & SANGUINETTI, EVELYN PA...",Won-Primary,REPUBLICAN,IL,2018,GOVERNOR / LIEUTENANT GOVERNOR,"RAUNER, BRUCE VINCENT","50,000,000.00",2016-12-20,1.00


Convert the contribution amount column to numeric (float) data type and the contribution date column to datetime data type.

In [7]:
contributions["amount"] = pd.to_numeric(contributions["amount"], errors="coerce")
contributions["date"] = pd.to_datetime(contributions["date"], errors="coerce")

In [8]:
contributions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8679160 entries, 0 to 8679159
Data columns (total 10 columns):
candidate          object
election_status    object
party              object
state              object
year               int64
office             object
contributor        object
amount             float64
date               datetime64[ns]
in_out_state       float64
dtypes: datetime64[ns](1), float64(2), int64(1), object(6)
memory usage: 662.2+ MB


Rename the categories in the in-vs.-out-of-state column.

In [9]:
# 0 = out-of-state, 1 = in-state, 2 = unknown
contributions["in_out_state"] = contributions["in_out_state"].replace({0: "out-of-state", 1: "in-state", 2: "unknown"})
contributions.head(1)

,candidate,election_status,party,state,year,office,contributor,amount,date,in_out_state
0,"RAUNER, BRUCE VINCENT & SANGUINETTI, EVELYN PA...",Won-Primary,REPUBLICAN,IL,2018,GOVERNOR / LIEUTENANT GOVERNOR,"RAUNER, BRUCE VINCENT","50,000,000.00",2016-12-20,in-state


Filter out unitemized donations as it is impossible to determine where those contributions originated.

In [10]:
contributions = contributions[contributions["contributor"] != "UNITEMIZED DONATIONS"]
contributions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8551014 entries, 0 to 8679159
Data columns (total 10 columns):
candidate          object
election_status    object
party              object
state              object
year               int64
office             object
contributor        object
amount             float64
date               datetime64[ns]
in_out_state       object
dtypes: datetime64[ns](1), float64(1), int64(1), object(7)
memory usage: 717.6+ MB


Create a standardized office column.

In [11]:
contributions["standardized_office"] = np.where(contributions["office"].str.contains("governor", case=False), "GOVERNOR/LIEUTENANT GOVERNOR",
                                       np.where(contributions["office"].str.contains("senate", case=False), "STATE SENATE",
                                       np.where(contributions["office"].str.contains("house", case=False), "STATE HOUSE", "")))
contributions.head(1)

/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,candidate,election_status,party,state,year,office,contributor,amount,date,in_out_state,standardized_office
0,"RAUNER, BRUCE VINCENT & SANGUINETTI, EVELYN PA...",Won-Primary,REPUBLICAN,IL,2018,GOVERNOR / LIEUTENANT GOVERNOR,"RAUNER, BRUCE VINCENT","50,000,000.00",2016-12-20,in-state,GOVERNOR/LIEUTENANT GOVERNOR


Filter the data by election year.

In [13]:
contributions_18 = contributions[contributions["year"] == 2018]
contributions_14 = contributions[contributions["year"] == 2014]
contributions_10 = contributions[contributions["year"] == 2010]

## Calculate a cut-off point for prior election cycles

Our next task is to determine a data cut-off point for prior election cycles so we can make accurate comparisons across cycles.

Extract the month and year from the contribution date column for 2018 election cycle data.

In [14]:
contributions_18["month"] = contributions_18["date"].dt.to_period("M")
contributions_18.info()

/home/joe/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


<class 'pandas.core.frame.DataFrame'>
Int64Index: 2135379 entries, 0 to 2164414
Data columns (total 12 columns):
candidate              object
election_status        object
party                  object
state                  object
year                   int64
office                 object
contributor            object
amount                 float64
date                   datetime64[ns]
in_out_state           object
standardized_office    object
month                  object
dtypes: datetime64[ns](1), float64(1), int64(1), object(9)
memory usage: 211.8+ MB


Group the contributions by state and month.

In [15]:
grouped_by_month = contributions_18.groupby(["state", "month"])["amount"].sum().reset_index()
grouped_by_month.head(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1436 entries, 0 to 1435
Data columns (total 3 columns):
state     1436 non-null object
month     1436 non-null object
amount    1436 non-null float64
dtypes: float64(1), object(2)
memory usage: 33.7+ KB


Because we eventually want to use each state's month column as the cut-off date for contributions, we need to add a day to the month and the year and then convert the column into datetime data type.

In [16]:
grouped_by_month["month"] = grouped_by_month["month"].astype(str) + "-28" # No month has fewer than 28 days
grouped_by_month["month"] = pd.to_datetime(grouped_by_month["month"], errors="coerce")
grouped_by_month.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1436 entries, 0 to 1435
Data columns (total 3 columns):
state     1436 non-null object
month     1436 non-null datetime64[ns]
amount    1436 non-null float64
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 33.7+ KB


We know some of the contribution dates are wrong. We know this because some of the dates occur in the future and, unless we've got some time travelling campaign donors, these are data entry errors. To eliminate this noise, we will filter out months after August 2018.

In [17]:
grouped_by_month = grouped_by_month[grouped_by_month["month"] <= "2018-08-28"]

Return the most recent month of contributions for each state.

In [18]:
latest_month = grouped_by_month.groupby("state")["month"].max().reset_index()
latest_month.rename(columns={"month": "latest_month"}, inplace=True)
latest_month

,state,latest_month
0,AK,2018-07-28
1,AL,2018-07-28
2,AR,2018-03-28
3,AZ,2017-12-28
4,CA,2018-07-28
5,CO,2018-07-28
6,CT,2018-07-28
7,FL,2018-08-28
8,GA,2018-07-28
9,HI,2018-07-28


## Apply the cut-off date to the 2014 and 2010 election cycles' data.

Join the table of the 2018 cycle's latest contribution months with the 2014 and 2010 contribution-level data.

In [19]:
contributions_14 = contributions_14.merge(latest_month, on="state")
contributions_10 = contributions_10.merge(latest_month, on="state")

Convert the year in the latest month column to its equivalent in the relevant election cycle.

In [ ]:
%%notify
# 2017 = 2013, 2018 = 2014
contributions_14["latest_month"] = contributions_14["latest_month"].mask(contributions_14["latest_month"].dt.year == 2017, contributions_14["latest_month"] + pd.offsets.DateOffset(year=2013))
contributions_14["latest_month"] = contributions_14["latest_month"].mask(contributions_14["latest_month"].dt.year == 2018, contributions_14["latest_month"] + pd.offsets.DateOffset(year=2014))
# 2017 = 2009, 2018 = 2010
contributions_10["latest_month"] = contributions_10["latest_month"].mask(contributions_10["latest_month"].dt.year == 2017, contributions_10["latest_month"] + pd.offsets.DateOffset(year=2009))
contributions_10["latest_month"] = contributions_10["latest_month"].mask(contributions_10["latest_month"].dt.year == 2018, contributions_10["latest_month"] + pd.offsets.DateOffset(year=2010))

/home/joe/anaconda3/lib/python3.6/site-packages/pandas/core/indexes/datetimes.py:962: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex
  "or DatetimeIndex", PerformanceWarning)


Filter the data to eliminate contributions after the 2018 cycle's latest contribution month in each state.

In [ ]:
contributions_14 = contributions_14[contributions_14["date"] <= contributions_14["latest_month"]]
contributions_10 = contributions_10[contributions_10["date"] <= contributions_10["latest_month"]]

## Export the data

In [ ]:
contributions_18.to_csv("data/contributions_18.csv", index=False)
contributions_14.to_csv("data/contributions_14.csv", index=False)
contributions_10.to_csv("data/contributions_10.csv", index=False)